In [ ]:
import numpy as np
import pandas as pd

In [ ]:
X_train =pd.read_csv("uci_train.csv")
X_test  =pd.read_csv("uci_test.csv")

In [ ]:
X_train

In [ ]:
X_train.columns

In [ ]:
X_test

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
fn=lambda x:x.split('-')[1]
X_train['Month']=X_train['Time'].map(fn)
X_test['Month']=X_test['Time'].map(fn)

sb.catplot(x="Month",hue="Target",data=X_train,kind='count')
plt.show()


plt.figure(figsize=(20,10))
sb.scatterplot(x=X_train.index,y=X_train['1'],hue=X_train['Target'])
sb.scatterplot(x=X_train.index,y=X_train['3'],hue=X_train['Target'])
sb.scatterplot(x=X_train.index,y=X_train['11'],hue=X_train['Target'])

In [ ]:
y = X_train['Target']
y
X_train.drop(columns = "Target",inplace =True)

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train.iloc[:,1:]=imp_mean.fit_transform(X_train.iloc[:,1:])


In [ ]:
Y_test= X_test["Target"]
Y_test
X_test = X_test.iloc[: , :-1]
X_test.iloc[:,1:]=imp_mean.transform(X_test.iloc[:,1:])

In [ ]:
Y_test

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sb

In [ ]:
X_train.columns

In [ ]:
X_train.isnull().any()

In [ ]:
pd.tools.plotting.scatter_matrix(X_train.loc[:, "2":"6"], diagonal="kde")
plt.tight_layout()
plt.show()


In [ ]:
columns=X_train.columns
drop_col=[]
for i in columns:
  if(len(X_train[i].unique())==1):
    drop_col.append(i)

In [ ]:
X_train.drop(columns=drop_col,inplace=True)
X_test.drop(columns=drop_col,inplace=True)

In [ ]:
X_train.columns

In [ ]:
X_train.drop(columns= "Time", inplace = True)
X_test.drop(columns= "Time", inplace = True)

In [ ]:
Xcolumns=X_train.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_train=pd.DataFrame(X_train,columns=Xcolumns)
X_test = sc_x.transform(X_test)
X_test=pd.DataFrame(X_test,columns=Xcolumns)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(
        X_train, y, test_size = 0.25)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=250,C=0.1)
classifier.fit(X_train, y)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
#y_testing=pd.read_csv('y_test.csv')

In [ ]:
y_train_pred = classifier.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print ("Accuracy Score : ", accuracy_score(y_train_pred, y))
print("Precision Score : ",precision_score(y_train_pred, y))
print("Recall Score : ",recall_score(y_train_pred, y))
print("F1 Score : ",f1_score(y_train_pred, y))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)

print ("Confusion Matrix : \n", cm)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print ("Accuracy : ", accuracy_score(Y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100)
rf_clf.fit(xtrain, ytrain)

In [ ]:
fimp={'f_name':[],'f_imp':[]}
for feature, importance in zip(X_train.columns, rf_clf.feature_importances_):
    fimp['f_name'].append(feature)
    fimp['f_imp'].append(importance)

In [ ]:
fimp=pd.DataFrame.from_dict(fimp)
fimp.sort_values(by=['f_imp'],ascending=[0],inplace=True)

In [ ]:
not_imp_feature=fimp[fimp['f_imp']==0].f_name.values

In [ ]:
extracted_X_train=X_train.drop(columns=not_imp_feature)

In [ ]:
extracted_X_test=X_test.drop(columns=not_imp_feature)

In [ ]:
extracted_X_test.dtypes



In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(
        extracted_X_train, y, test_size = 0.3, random_state = 24)

In [ ]:
y.dtypes

In [ ]:
param_grid = {'C': [0.1,0.5],
              'max_iter':[500,1000],
              'solver':['liblinear'],
              'penalty':['l2']
              }

In [ ]:
plt.figure(figsize=(10,10))
#sb.scatterplot(x=y,y=extracted_X_test['X10'],hue=y)
sb.kdeplot(data=extracted_X_test,x='2', hue=y,fill=True)

In [ ]:
sb.barplot(data=extracted_X_test,x='15', hue=y,fill=True)

In [ ]:
logreg_cv = GridSearchCV(classifier, param_grid, cv = 5)
logreg_cv.fit(xtrain,ytrain)

In [ ]:
logreg_cv.best_params_

In [ ]:
final_model=logreg_cv.best_estimator_

In [ ]:
rf_y_pred=final_model.predict(extracted_X_test)

In [ ]:

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

print ("Accuracy : ", accuracy_score(Y_test, rf_y_pred))
print(precision_score(Y_test,rf_y_pred))
print(recall_score(Y_test,rf_y_pred))
print(f1_score(Y_test,rf_y_pred))

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100)
rf_clf.fit(extracted_X_train, y)